In [ ]:
df_ = df[['title', 'plot_synopsis']][0:10]
df_ = df_.reset_index()
df_.columns = ['_id', 'title', 'plot_synopsis']
df_=df_.to_dict(orient='records')
df_

In [ ]:
#encode df directly to json
from vectorhub.encoders.text.sentence_transformers import SentenceTransformer2Vec
model = SentenceTransformer2Vec("bert-base-uncased")
df_json = model.encode_documents(documents=json_files[0:1000], fields=['product_name'])
df_json

## upload to relevanceai

In [1]:
import relevanceai

#ea126145236030e36199:VDl2U2pIMEJ0TS1TY20ycEo3MzY6dHhDYklfeEVTNnUwRHNzaXBMTDhrdw NEW ID
project = "michelangioloma"
api_key = "TDFvdm9Yd0J1M3VVTEcyQTV3VWs6b3RnRXJnUW9SQ2kwMVVJWVZ2VzVtZw"
client = relevanceai.Client(project, api_key)

In [ ]:
#encoded df to json for uploading
import pandas as pd

df_movies = pd.read_csv('datasets/mpst_drop_encoded.csv')
df_movies

list1 = list()
for index in range(len(df_movies['title'])):
    if index % 100 == 0:
        print(index)
    dict1 = dict()
    title = df_movies['title'].iloc[index]
    vectors = list(df_movies.drop('title', axis=1).values[index])
    dict1['_id'] = index
    dict1['title'] = title
    dict1['movie_transformers_vector_'] = vectors
    list1.append(dict1)
list1

client.datasets.bulk_insert(dataset_id='mpst_movie_dataset', documents=list1)

In [ ]:
client.datasets.schema('mpst_movie_dataset')

## movie recommender

In [ ]:
import relevanceai

connection = relevanceai.api.endpoints.search.Search("ea126145236030e36199", 'VDl2U2pIMEJ0TS1TY20ycEo3MzY6dHhDYklfeEVTNnUwRHNzaXBMTDhrdw')

In [ ]:
response = connection.traditional(
    'mpst_movie_dataset',
    text='...tick... tick... tick..',
    fields=['title'],
    include_vector=True
)
response

In [ ]:
response = connection.vector(
    'mpst_movie_dataset',
    multivector_query=[
            {"vector": list1[0]['movie_transformers_vector_'], "fields": ['movie_transformers_vector_']}
    ]
)
response

In [ ]:
#I input a string present in the dataset, API finds the closest match, then returns a nns
def recommend_movies(str1):
    response_trad = connection.traditional(
        'mpst_movie_dataset',
        text=str1,
        fields=['title'],
        include_vector=True
    )

    response_vect = connection.vector(
        'mpst_movie_dataset',
        multivector_query=[
                {"vector": response_trad['results'][0]['movie_transformers_vector_'], "fields": ['movie_transformers_vector_']}
        ],
        page_size=3
    )

    for recc in response_vect['results'][1:]:
        print(recc['title'])

recommend_movies('...tick... tick... tick..')